In [2]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Codec:
    def serialize(self, root):
        def preorder(node):
            if not node:
                return ['#']
            return [str(node.val)] + preorder(node.left) + preorder(node.right)
        return ','.join(preorder(root))

    def deserialize(self, data):
        def build(nodes):
            val = next(nodes)
            if val == '#':
                return None
            node = TreeNode(int(val))
            node.left = build(nodes)
            node.right = build(nodes)
            return node
        return build(iter(data.split(',')))

codec = Codec()
root = TreeNode(1, TreeNode(2), TreeNode(3, TreeNode(4), TreeNode(5)))

serialized = codec.serialize(root)
print("Serialized:", serialized)

deserialized = codec.deserialize(serialized)
print("Deserialized Root:", deserialized.val) 

Serialized: 1,2,#,#,3,4,#,#,5,#,#
Deserialized Root: 1


In [3]:
from collections import deque

def bfs(rGraph, s, t, parent):
    visited = [False] * len(rGraph)
    queue = deque([s])
    visited[s] = True
    while queue:
        u = queue.popleft()
        for v, capacity in enumerate(rGraph[u]):
            if not visited[v] and capacity > 0:
                queue.append(v)
                visited[v] = True
                parent[v] = u
    return visited[t]

def ford_fulkerson(graph, source, sink):
    rGraph = [row[:] for row in graph]
    parent = [-1] * len(graph)
    max_flow = 0
    while bfs(rGraph, source, sink, parent):
        path_flow = float('inf')
        v = sink
        while v != source:
            u = parent[v]
            path_flow = min(path_flow, rGraph[u][v])
            v = parent[v]
        max_flow += path_flow
        v = sink
        while v != source:
            u = parent[v]
            rGraph[u][v] -= path_flow
            rGraph[v][u] += path_flow
            v = parent[v]
    return max_flow

graph = [[0, 16, 13, 0, 0, 0],
         [0, 0, 10, 12, 0, 0],
         [0, 4, 0, 0, 14, 0],
         [0, 0, 9, 0, 0, 20],
         [0, 0, 0, 7, 0, 4],
         [0, 0, 0, 0, 0, 0]]

print("Maximum Flow:", ford_fulkerson(graph, 0, 5)) 

Maximum Flow: 23


In [4]:
def edit_distance(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0]*(n+1) for _ in range(m+1)]

    for i in range(m+1):
        for j in range(n+1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
    return dp[m][n]

s1 = "kitten"
s2 = "sitting"
print("Edit Distance:", edit_distance(s1, s2)) 

Edit Distance: 3


In [5]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def kth_smallest(root, k):
    def inorder(node):
        if not node:
            return []
        return inorder(node.left) + [node.val] + inorder(node.right)
    
    return inorder(root)[k-1]

root = TreeNode(5)
root.left = TreeNode(3, TreeNode(2), TreeNode(4))
root.right = TreeNode(6)

k = 3
print("K-th Smallest:", kth_smallest(root, k)) 

K-th Smallest: 4


In [6]:
def max_product(nums):
    if not nums:
        return 0

    max_prod = min_prod = result = nums[0]
    for num in nums[1:]:
        if num < 0:
            max_prod, min_prod = min_prod, max_prod
        max_prod = max(num, max_prod * num)
        min_prod = min(num, min_prod * num)
        result = max(result, max_prod)
    return result

nums = [2, 3, -2, 4]
print("Max Subarray Product:", max_product(nums)) 

Max Subarray Product: 6


In [7]:
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path:
            newpaths = find_all_paths(graph, node, end, path)
            for p in newpaths:
                paths.append(p)
    return paths

graph = {
    'A': ['B', 'C'],
    'B': ['C', 'D'],
    'C': ['D'],
    'D': []
}
start = 'A'
end = 'D'
print("All Paths:", find_all_paths(graph, start, end))


All Paths: [['A', 'B', 'C', 'D'], ['A', 'B', 'D'], ['A', 'C', 'D']]


In [9]:
class CustomSerializer:
    def serialize(self, data):
        if isinstance(data, dict):
            items = []
            for key, value in data.items():
                items.append(f"{key}:{self.serialize(value)}")
            return '{' + ', '.join(items) + '}'
        elif isinstance(data, list):
            return '[' + ', '.join(self.serialize(item) for item in data) + ']'
        elif isinstance(data, str):
            return f'"{data}"'
        else:
            return str(data)

    def deserialize(self, s):
        def parse_value(tokens):
            token = tokens.pop(0)
            if token == '{':
                obj = {}
                while tokens[0] != '}':
                    key = tokens.pop(0)
                    tokens.pop(0)  # colon
                    value = parse_value(tokens)
                    obj[key] = value
                    if tokens[0] == ',':
                        tokens.pop(0)
                tokens.pop(0)  # closing brace
                return obj
            elif token == '[':
                lst = []
                while tokens[0] != ']':
                    lst.append(parse_value(tokens))
                    if tokens[0] == ',':
                        tokens.pop(0)
                tokens.pop(0)  # closing bracket
                return lst
            elif token.startswith('"'):
                return token.strip('"')
            elif token.isdigit():
                return int(token)
            else:
                return token

        def tokenize(s):
            tokens = []
            token = ''
            in_string = False
            for char in s:
                if char == '"':
                    in_string = not in_string
                    token += char
                elif char in '{}[],:':
                    if token.strip():
                        tokens.append(token.strip())
                        token = ''
                    tokens.append(char)
                elif char == ' ' and not in_string:
                    continue
                else:
                    token += char
            if token.strip():
                tokens.append(token.strip())
            return tokens

        tokens = tokenize(s)
        return parse_value(tokens)

    def validate_schema(self, data, schema):
        if not isinstance(data, dict) or not isinstance(schema, dict):
            return False
        for key, expected_type in schema.items():
            if key not in data:
                print(f"Missing key: {key}")
                return False
            value = data[key]
            if expected_type == 'int' and not isinstance(value, int):
                print(f"Key '{key}' should be int but got {type(value).__name__}")
                return False
            elif expected_type == 'str' and not isinstance(value, str):
                print(f"Key '{key}' should be str but got {type(value).__name__}")
                return False
            elif expected_type == 'dict' and not isinstance(value, dict):
                print(f"Key '{key}' should be dict but got {type(value).__name__}")
                return False
            elif isinstance(expected_type, dict):
                if not self.validate_schema(value, expected_type):
                    return False
        return True

tool = CustomSerializer()

# Sample data
data = {
    'name': "John",
    'age': 30,
    'address': {
        'city': "Chennai",
        'zip': 600001
    }
}

# Serialize
serialized = tool.serialize(data)
print("Serialized:", serialized)

# Deserialize
deserialized = tool.deserialize(serialized)
print("Deserialized:", deserialized)

# Schema Validation
schema = {
    'name': 'str',
    'age': 'int',
    'address': {
        'city': 'str',
        'zip': 'int'
    }
}
print("Is Valid:", tool.validate_schema(deserialized, schema))


Serialized: {name:"John", age:30, address:{city:"Chennai", zip:600001}}
Deserialized: {'name': 'John', 'age': 30, 'address': {'city': 'Chennai', 'zip': 600001}}
Is Valid: True
